In [1]:
import os
import requests
import zipfile
import shutil

# 로그인 위치
url = 'https://tagtog.net/-login'

# 다운로드 위치
file_url = 'https://tagtog.net/nannullna/this-is-real/-downloads/dataset-as-anndoc'
zip_file = 'download.zip'

if os.path.exists(zip_file):
    os.remove(zip_file)
    
# 로그인 정보
login_info = {
    'loginid': '', # 아이디 입력
    'password': '' # 비밀번호 입력
}

# 로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)
    
    with open(zip_file, 'wb') as output:
        output.write(r.content)
        
# 압축 파일 풀기
folder_path = './tagtog_result'

zip_ = zipfile.ZipFile(zip_file)

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)

zip_.extractall(folder_path)

os.remove(zip_file)

In [36]:
import json
import glob
import os

# 폴더 경로
root_path = os.path.join(folder_path, 'this-is-real')

json_root_path = os.path.join(root_path, 'ann.json/master/pool')
html_root_path = os.path.join(root_path, 'plain.html/pool')

target_folder = 'test/jeonju_hyanggyo'

json_path = os.path.join(json_root_path, target_folder)
html_path = os.path.join(html_root_path, target_folder)

def get_unique_file_name(file_name):
    return file_name[:file_name[:file_name.rfind('.')].rfind('.')]

# 파일명 목록
file_list = [get_unique_file_name(file) for file in os.listdir(json_path)]

# 파일 목록
json_file_list = os.listdir(json_path)
html_file_list = os.listdir(html_path)

files = {}
for file in file_list:
    files[file] = {'json': '', 'html': ''}

for json_file in json_file_list:
    files[get_unique_file_name(json_file)]['json'] = os.path.join(json_path, json_file)
    
for html_file in html_file_list:
    files[get_unique_file_name(html_file)]['html'] = os.path.join(html_path, html_file)

# annotation_legend
annotation_legend = os.path.join(root_path, 'annotations-legend.json')
with open(annotation_legend, 'r') as f:
    annotation_legend = json.load(f)

In [86]:
import pandas as pd
from bs4 import BeautifulSoup

data = {
    'title': [],
    'sentence': [],
    'sentence_with_entity': [],
    'subject_entity_word': [],
    'subject_entity_start_idx': [],
    'subject_entity_end_idx': [],
    'subject_entity_type': [],
    'object_entity_word': [],
    'object_entity_start_idx': [],
    'object_entity_end_idx': [],
    'object_entity_type': [],    
}

for i, key in enumerate(files.keys()):
    # get title and sentence information from html file
    with open(files[key]['html'], 'r') as f:
        html_obj = f.read()
        
    bs_obj = BeautifulSoup(html_obj, 'html.parser')
    title, sentence = [obj.text for obj in bs_obj.select('pre')]

    data['title'].append(title)
    data['sentence'].append(sentence)

    # get entity information from json file
    with open(files[key]['json'], 'r') as f:
        json_obj = json.load(f)
        
    subject_entity = {'word': '', 'start_idx': 0, 'end_idx': 0, 'type': ''}
    object_entity = {'word': '', 'start_idx': 0, 'end_idx': 0, 'type': ''}
    entities = json_obj['entities']
    for entity in entities:
        e_info, e_type = annotation_legend[entity['classId']].split('_')
        if e_info == 'subj':
            data['subject_entity_word'].append(entity['offsets'][0]['text'])
            data['subject_entity_start_idx'].append(entity['offsets'][0]['start'])
            data['subject_entity_end_idx'].append(entity['offsets'][0]['start'] + len(entity['offsets'][0]['text']) - 1)
            data['subject_entity_type'].append(e_type)
        else:
            data['object_entity_word'].append(entity['offsets'][0]['text'])
            data['object_entity_start_idx'].append(entity['offsets'][0]['start'])
            data['object_entity_end_idx'].append(entity['offsets'][0]['start'] + len(entity['offsets'][0]['text']) - 1)
            data['object_entity_type'].append(e_type)

    # get sentence with entities information
    sentence_w_entity = sentence
    subject_entity = {
        'symbol': '$$',
        'start': data['subject_entity_start_idx'][i],
        'end': data['subject_entity_end_idx'][i]
    }
    object_entity = {
        'symbol': '@@',
        'start': data['object_entity_start_idx'][i],
        'end': data['object_entity_end_idx'][i]
    }
    entities = sorted([subject_entity, object_entity], key=lambda x: x['start'], reverse=True)
    for entity in entities:
        b_str = sentence_w_entity[:entity['start']]
        e_str = sentence_w_entity[entity['start']:entity['end']+1]
        a_str = sentence_w_entity[entity['end']+1:]
        sentence_w_entity = b_str + entity['symbol'] + e_str + entity['symbol'] + a_str    
    data['sentence_with_entity'].append(sentence_w_entity)

df = pd.DataFrame(data)

In [87]:
df.head(5)

,title,sentence,sentence_with_entity,subject_entity_word,subject_entity_start_idx,subject_entity_end_idx,subject_entity_type,object_entity_word,object_entity_start_idx,object_entity_end_idx,object_entity_type
0,전주향교,전주향교 건물도 인민위원회의 사무실이며 마초지로 쓰이며 훼손되었다.,$$전주향교$$ 건물도 @@인민위원회의 사무실@@이며 마초지로 쓰이며 훼손되었다.,전주향교,0,3,LOC,인민위원회의 사무실,9,18,LOC
1,전주향교,"1992년 12월 24일, 한국의 당시 문화부는 전주향교 일원을 사적 제379호로 ...","1992년 12월 24일, 한국의 당시 문화부는 $$전주향교$$ 일원을 @@사적 제...",전주향교,27,30,LOC,사적 제379호,36,43,POH
2,전주향교,"전주향교의 원래의 위치는 전주시 풍남동이었으나, 이곳 남쪽에 태종(太宗) 10년(1...","$$전주향교$$의 원래의 위치는 전주시 풍남동이었으나, 이곳 남쪽에 태종(太宗) 1...",전주향교,0,3,LOC,황학대(黃鶴臺),196,203,LOC
3,전주향교,전주향교는 원래 고려(高麗) 공민왕(恭愍王) 3년(1354년)에 지어졌다고 전하고 있다.,$$전주향교$$는 원래 고려(高麗) @@공민왕(恭愍王)@@ 3년(1354년)에 지어...,전주향교,0,3,LOC,공민왕(恭愍王),16,23,PER
4,전주향교,전주향교는 1992년 12월 24일 사적 제379호로 지정되었다.,$$전주향교$$는 1992년 12월 24일 @@사적 제379호@@로 지정되었다.,전주향교,0,3,LOC,사적 제379호,20,27,POH
